In [1]:
import os
import csv
import time
import torch
import whisper
import ffmpeg
from collections import deque

In [2]:
# Убедитесь, что имеется доступ к CUDA
device = "cuda" if torch.cuda.is_available() else "cpu"


In [3]:
# Загрузите модель
model = whisper.load_model("small", device=device)


/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


In [4]:
# Путь к папке с видео
video_folder = "/home/user1/hak/video/videos_2"

# Файл для записи результата
output_file = "video_transcriptions.csv"

# Список всех mp4 файлов в папке, отсортированных по размеру
video_files = sorted(
    [(f, os.path.getsize(os.path.join(video_folder, f))) for f in os.listdir(video_folder) if f.endswith('.mp4')],
    key=lambda x: x[1]
)


In [5]:
# Проверяем, какие видео уже были обработаны
processed_videos = set()
if os.path.exists(output_file):
    with open(output_file, mode='r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            processed_videos.add(row['id'])


In [6]:
total_videos = len(video_files)  # Общее количество видео

# Открываем CSV файл для записи, добавляя новые результаты
with open(output_file, mode='a', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['id', 'text']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Пишем заголовки только если файл был создан
    if not processed_videos:
        writer.writeheader()

    # Хранение времени обработки для последних N видео
    recent_times = deque(maxlen=5)
    total_processing_time = 0

    for index, (video_file, size) in enumerate(video_files, start=1):
        video_id = os.path.splitext(video_file)[0]

        if video_id in processed_videos:
            print(f"Skipping {video_file}, already processed.")
            continue

        video_path = os.path.join(video_folder, video_file)
        temp_audio_path = "temp_audio.wav"

        # Извлечение первых 2.5 минут аудио
        try:
            ffmpeg.input(video_path, t=150).output(temp_audio_path, acodec='pcm_s16le', ac=1, ar='16000').run(quiet=True, overwrite_output=True)
        except Exception as e:
            print(f"Error processing {video_file}: {e}")
            continue

        # Обработка видео файла
        print(f"Processing {index}/{total_videos}: {video_file}...")

        video_start_time = time.time()  # Начало времени обработки видео

        # Получаем текст с аудио
        result = model.transcribe(temp_audio_path)

        # Время, потраченное на обработку видео
        video_processing_time = time.time() - video_start_time
        recent_times.append(video_processing_time)
        total_processing_time += video_processing_time

        # Пишем результат в CSV файл
        writer.writerow({'id': video_id, 'text': result['text']})

        # Удаляем временный аудиофайл
        if os.path.exists(temp_audio_path):
            os.remove(temp_audio_path)

        # Вычисляем процент выполнения и оставшееся время
        average_time_per_video = sum(recent_times) / len(recent_times)
        estimated_total_time = average_time_per_video * total_videos
        estimated_remaining_time = estimated_total_time - total_processing_time

        percent_complete = (index / total_videos) * 100

        print(f"Completed {index}/{total_videos}.")
        print(f"Video processing time: {video_processing_time:.2f} seconds.")
        print(f"Estimated time remaining: {estimated_remaining_time:.2f} seconds.")

    print("Обработка всех видео завершена.")


Skipping 92f09d5f55fc50dd70d741f8c1aec93c.mp4, already processed.
Skipping f76723662f79bca0d45acfe491a5aeee.mp4, already processed.
Skipping dd7d4e24343a105bd91a3ccd698c8908.mp4, already processed.
Skipping e714d955948cdfb7c8edd5d8325a5ff6.mp4, already processed.
Skipping c618d967758fcd6fe731e3bd4ba986f9.mp4, already processed.
Skipping f93266d3ac87373213a74ae2a7dcb94b.mp4, already processed.
Skipping 5fbc221f3df6cff73f2d272ac16aeea3.mp4, already processed.
Skipping 54056fbd12590754a008667afc7751ec.mp4, already processed.
Skipping 29f12c52655b55c5f0f114a79215785d.mp4, already processed.
Skipping 9b8a5b893d1c620856d550b030161178.mp4, already processed.
Skipping 86a76fa0af1e08aa1911c91cca9a740f.mp4, already processed.
Skipping 9143ffe5052ed8b7a482ce347cc69397.mp4, already processed.
Skipping d8d14605a042a1d7d7c8493d7ca70e08.mp4, already processed.
Skipping 099fd80077931649d5e71fa95cdbf42b.mp4, already processed.
Skipping e082d4ce348a0b2b18804eabdc5c2a33.mp4, already processed.
Skipping b